# Visualización 3D de Tres Pozos del Campo Sacha

**Nombres anónimos:** Alfa 1, Alfa 2, Alfa 3  
**Autor:** Patricia García  
**Fecha:** Enero 2025

---

## ¿Qué vamos a hacer en este cuaderno?

Vamos a crear una visualización 3D interactiva de tres pozos petroleros del Campo Sacha en Ecuador. La visualización mostrará:

1. **Las trayectorias completas** de cada pozo desde la superficie hasta el fondo
2. **Los puntos de casing** donde se instalaron las tuberías de revestimiento (13 3/8", 9 5/8" y 7")
3. **Las formaciones reservorio** Hollín Superior e Inferior, que son los objetivos principales de estos pozos

La idea es que puedas rotar, hacer zoom y explorar la geometría de los tres pozos en un espacio 3D interactivo.

---

## Conceptos clave antes de empezar

Si eres nuevo en esto de pozos direccionales, aquí van algunos conceptos que te ayudarán a entender lo que estamos haciendo:

**MD (Measured Depth):** Es la profundidad medida a lo largo del pozo. Es como si pusieras una cinta métrica que sigue toda la curva del pozo desde la superficie hasta el fondo.

**TVD (True Vertical Depth):** Es la profundidad vertical real. Es la distancia en línea recta desde la superficie hasta un punto del pozo, como si tiraras una plomada.

**NS y EW:** Son las coordenadas de desplazamiento. NS es Norte-Sur (positivo hacia el norte) y EW es Este-Oeste (positivo hacia el este). Estas coordenadas nos dicen qué tanto se ha desviado el pozo horizontalmente desde su punto de inicio.

**Casing:** Son las tuberías de acero que se instalan en el pozo para proteger las formaciones y controlar presiones. Se instalan en etapas: primero el casing superficial (más grande), luego el intermedio, y finalmente el de producción (más pequeño).

**Formación Hollín:** Es la roca reservorio principal del Oriente Ecuatoriano. Contiene petróleo y es el objetivo de la mayoría de los pozos. Se divide en Hollín Superior e Inferior.

## Celda 1: Importar las librerías que necesitamos

Antes de empezar a trabajar con datos, necesitamos importar las herramientas de Python que vamos a usar:

- **pandas:** Es como Excel pero en Python. Nos ayuda a cargar y manipular tablas de datos.
- **plotly:** Es una librería para hacer gráficos interactivos en 3D. Nos permite crear visualizaciones que puedes rotar con el mouse.
- **numpy:** Es una librería para hacer cálculos matemáticos rápidos con números.

In [ ]:
# Importamos pandas para trabajar con tablas de datos
import pandas as pd

# Importamos plotly para hacer gráficos 3D interactivos
import plotly.graph_objects as go

# Importamos numpy para hacer cálculos matemáticos
import numpy as np

# Este print nos confirma que todo se importó correctamente
print("✅ Librerías importadas correctamente")
print("   Ya podemos empezar a trabajar con los datos")

## Celda 2: Cargar los datos de los tres pozos

Ahora vamos a cargar los archivos CSV que contienen los datos de los surveys de cada pozo. Un survey es básicamente una tabla que nos dice, para cada profundidad del pozo, cuál es su posición exacta en el espacio 3D.

Cada archivo CSV tiene estas columnas:
- **MD:** Profundidad medida en pies
- **Inc:** Inclinación en grados (0° es vertical, 90° es horizontal)
- **Azim:** Azimut en grados (dirección de la brújula: 0° es norte, 90° es este, etc.)
- **TVD:** Profundidad vertical verdadera en pies
- **NS:** Desplazamiento Norte-Sur en pies
- **EW:** Desplazamiento Este-Oeste en pies

In [ ]:
# Cargamos los datos del primer pozo (Alfa 1)
# pd.read_csv() lee un archivo CSV y lo convierte en una tabla que podemos usar
alfa1 = pd.read_csv('survey_alfa1_377D.csv')

# Cargamos el segundo pozo (Alfa 2)
alfa2 = pd.read_csv('survey_alfa2_378D.csv')

# Cargamos el tercer pozo (Alfa 3)
alfa3 = pd.read_csv('survey_alfa3_208D.csv')

# Ahora imprimimos información sobre cada pozo para verificar que se cargaron bien
print("="*70)
print("INFORMACIÓN DE LOS TRES POZOS")
print("="*70)

print("\n📍 POZO ALFA 1:")
print(f"   Puntos de survey: {len(alfa1)}")
print(f"   Profundidad total (MD): {alfa1['MD'].max():.0f} pies")
print(f"   Profundidad vertical (TVD): {alfa1['TVD'].max():.0f} pies")

print("\n📍 POZO ALFA 2:")
print(f"   Puntos de survey: {len(alfa2)}")
print(f"   Profundidad total (MD): {alfa2['MD'].max():.0f} pies")
print(f"   Profundidad vertical (TVD): {alfa2['TVD'].max():.0f} pies")

print("\n📍 POZO ALFA 3:")
print(f"   Puntos de survey: {len(alfa3)}")
print(f"   Profundidad total (MD): {alfa3['MD'].max():.0f} pies")
print(f"   Profundidad vertical (TVD): {alfa3['TVD'].max():.0f} pies")

print("\n✅ Los tres pozos se cargaron correctamente")

## Celda 3: Definir las profundidades de los casing points

Los casings (tuberías de revestimiento) se instalan a profundidades específicas durante la perforación del pozo. En el Campo Sacha, las profundidades típicas son:

- **13 3/8" (Surface Casing):** ~2,500 pies. Es el casing superficial que protege los acuíferos.
- **9 5/8" (Intermediate Casing):** ~8,500 pies. Protege formaciones intermedias.
- **7" (Production Casing):** ~10,300 pies. Es el casing final que llega hasta el reservorio.

Nota: Estos valores son aproximados y pueden variar entre pozos dependiendo de las condiciones geológicas que encuentren durante la perforación.

In [ ]:
# Definimos las profundidades aproximadas de cada casing
# Estas son profundidades medidas (MD) en pies

casing_13_38 = 2500   # Casing superficial de 13 3/8 pulgadas
casing_9_58 = 8500    # Casing intermedio de 9 5/8 pulgadas
casing_7 = 10300      # Casing de producción de 7 pulgadas

print("PROFUNDIDADES DE CASING DEFINIDAS:")
print(f"  • 13 3/8\" (Surface):      {casing_13_38:,} pies")
print(f"  • 9 5/8\"  (Intermediate): {casing_9_58:,} pies")
print(f"  • 7\"      (Production):   {casing_7:,} pies")

## Celda 4: Definir las profundidades de las formaciones Hollín

La Formación Hollín es el objetivo principal de estos pozos porque contiene el petróleo. Se divide en dos miembros:

- **Hollín Superior:** Es la parte superior del reservorio
- **Hollín Inferior:** Es la parte inferior, típicamente de mejor calidad

Las profundidades varían un poco entre pozos debido a la estructura geológica, pero típicamente el Hollín se encuentra entre 10,100 y 10,300 pies de profundidad en el Campo Sacha.

In [ ]:
# Definimos las profundidades aproximadas de las formaciones
# Estas son profundidades medidas (MD) en pies

hollin_superior = 10167  # Tope de Hollín Superior
hollin_inferior = 10212  # Tope de Hollín Inferior

print("PROFUNDIDADES DE FORMACIONES RESERVORIO:")
print(f"  • Hollín Superior: {hollin_superior:,} pies")
print(f"  • Hollín Inferior: {hollin_inferior:,} pies")
print(f"\n  Espesor de Hollín Superior: ~{hollin_inferior - hollin_superior} pies")

## Celda 5: Crear la figura 3D vacía

Ahora vamos a empezar a construir nuestra visualización 3D. Lo primero es crear una figura vacía de Plotly. Piensa en esto como preparar un lienzo en blanco donde vamos a ir pintando las trayectorias de los pozos.

La figura usa un sistema de coordenadas 3D donde:
- **Eje X:** Representa el desplazamiento Norte-Sur
- **Eje Y:** Representa el desplazamiento Este-Oeste
- **Eje Z:** Representa la profundidad (negativa porque va hacia abajo)

In [ ]:
# Creamos una figura vacía de Plotly
# Esta figura es como un contenedor donde vamos a ir agregando elementos
fig = go.Figure()

print("✅ Figura 3D creada")
print("   Ahora vamos a empezar a agregar los pozos y marcadores")

## Celda 6: Agregar la trayectoria del Pozo Alfa 1 (Rojo)

Vamos a agregar la primera trayectoria a nuestra figura. El Pozo Alfa 1 se dibujará como una línea roja que conecta todos los puntos del survey.

Para cada punto del survey, tenemos:
- La posición NS (coordenada X)
- La posición EW (coordenada Y)
- La profundidad TVD (coordenada Z, negativa porque va hacia abajo)

Conectando todos estos puntos en orden, obtenemos la trayectoria completa del pozo desde la superficie hasta el fondo.

In [ ]:
# Agregamos la trayectoria del Pozo Alfa 1
# go.Scatter3d() crea una línea en 3D

fig.add_trace(go.Scatter3d(
    x=alfa1['NS'],           # Coordenada X (Norte-Sur)
    y=alfa1['EW'],           # Coordenada Y (Este-Oeste)
    z=-alfa1['TVD'],         # Coordenada Z (Profundidad, negativa)
    mode='lines',            # Dibujamos solo una línea continua
    name='ALFA 1',           # Nombre que aparecerá en la leyenda
    line=dict(
        color='red',         # Color rojo para este pozo
        width=6              # Grosor de la línea
    ),
    # Cuando pases el mouse sobre la línea, mostrará esta información:
    hovertemplate='<b>POZO ALFA 1</b><br>MD: %{customdata:.0f} pies<extra></extra>',
    customdata=alfa1['MD']   # Datos extra para mostrar (MD en este caso)
))

print("✅ Trayectoria del Pozo Alfa 1 agregada (color rojo)")

## Celda 7: Agregar la trayectoria del Pozo Alfa 2 (Azul)

Ahora agregamos el segundo pozo con exactamente el mismo proceso, pero usando un color diferente (azul) para distinguirlo del primero.

In [ ]:
# Agregamos la trayectoria del Pozo Alfa 2
# El código es muy similar al anterior, solo cambia la fuente de datos y el color

fig.add_trace(go.Scatter3d(
    x=alfa2['NS'],           # Datos del pozo Alfa 2
    y=alfa2['EW'],
    z=-alfa2['TVD'],
    mode='lines',
    name='ALFA 2',
    line=dict(
        color='blue',        # Color azul para distinguirlo de Alfa 1
        width=6
    ),
    hovertemplate='<b>POZO ALFA 2</b><br>MD: %{customdata:.0f} pies<extra></extra>',
    customdata=alfa2['MD']
))

print("✅ Trayectoria del Pozo Alfa 2 agregada (color azul)")

## Celda 8: Agregar la trayectoria del Pozo Alfa 3 (Verde)

Finalmente, agregamos el tercer pozo en color verde.

In [ ]:
# Agregamos la trayectoria del Pozo Alfa 3

fig.add_trace(go.Scatter3d(
    x=alfa3['NS'],
    y=alfa3['EW'],
    z=-alfa3['TVD'],
    mode='lines',
    name='ALFA 3',
    line=dict(
        color='green',       # Color verde para el tercer pozo
        width=6
    ),
    hovertemplate='<b>POZO ALFA 3</b><br>MD: %{customdata:.0f} pies<extra></extra>',
    customdata=alfa3['MD']
))

print("✅ Trayectoria del Pozo Alfa 3 agregada (color verde)")
print("\n🎨 Las tres trayectorias ya están en la figura")

## Celda 9: Marcar los puntos de superficie

Ahora vamos a marcar el punto donde cada pozo empieza en la superficie. Todos los pozos comienzan en el punto (0, 0, 0) que representa el origen de coordenadas.

Usaremos un diamante dorado para hacer este punto bien visible.

In [ ]:
# Agregamos marcadores en la superficie para los tres pozos
# Como todos empiezan en el origen, ponemos tres marcadores en (0,0,0)

fig.add_trace(go.Scatter3d(
    x=[0, 0, 0],             # Tres pozos, todos en NS = 0
    y=[0, 0, 0],             # Todos en EW = 0
    z=[0, 0, 0],             # Todos en la superficie (TVD = 0)
    mode='markers+text',     # Mostramos marcadores Y texto
    name='SUPERFICIE',
    marker=dict(
        size=15,             # Tamaño del marcador
        color='gold',        # Color dorado para que destaque
        symbol='diamond'     # Forma de diamante
    ),
    text=['SUPERFICIE', 'SUPERFICIE', 'SUPERFICIE'],  # Etiqueta de texto
    textposition='top center'  # Posición del texto (arriba y centrado)
))

print("✅ Marcadores de superficie agregados")

## Celda 10: Agregar casing de 13 3/8" en los tres pozos

Ahora vamos a marcar dónde se instaló el casing superficial de 13 3/8" en cada uno de los tres pozos.

Para hacer esto, necesitamos encontrar el punto del survey que esté más cercano a la profundidad de 2,500 pies (que es donde definimos que está este casing). 

**¿Por qué no podemos usar exactamente 2,500 pies?** Porque los datos del survey no necesariamente tienen un punto exactamente a esa profundidad. Por ejemplo, puede haber un punto a 2,490 pies y el siguiente a 2,510 pies. Entonces buscamos cuál de esos dos está más cerca de 2,500.

El proceso es:
1. Calcular la diferencia entre cada MD del survey y 2,500
2. Encontrar cuál diferencia es la más pequeña (en valor absoluto)
3. Usar ese punto del survey

In [ ]:
# POZO ALFA 1 - Casing 13 3/8"
# Calculamos la diferencia absoluta entre cada MD y la profundidad del casing
diferencias_alfa1 = abs(alfa1['MD'] - casing_13_38)
# Encontramos el índice donde la diferencia es mínima
indice_minimo_alfa1 = diferencias_alfa1.idxmin()
# Obtenemos el punto completo del survey en ese índice
punto_alfa1_13 = alfa1.loc[indice_minimo_alfa1]

# POZO ALFA 2 - Casing 13 3/8"
diferencias_alfa2 = abs(alfa2['MD'] - casing_13_38)
indice_minimo_alfa2 = diferencias_alfa2.idxmin()
punto_alfa2_13 = alfa2.loc[indice_minimo_alfa2]

# POZO ALFA 3 - Casing 13 3/8"
diferencias_alfa3 = abs(alfa3['MD'] - casing_13_38)
indice_minimo_alfa3 = diferencias_alfa3.idxmin()
punto_alfa3_13 = alfa3.loc[indice_minimo_alfa3]

# Ahora agregamos estos tres puntos a la figura
fig.add_trace(go.Scatter3d(
    # Usamos las coordenadas de cada punto que encontramos
    x=[punto_alfa1_13['NS'], punto_alfa2_13['NS'], punto_alfa3_13['NS']],
    y=[punto_alfa1_13['EW'], punto_alfa2_13['EW'], punto_alfa3_13['EW']],
    z=[-punto_alfa1_13['TVD'], -punto_alfa2_13['TVD'], -punto_alfa3_13['TVD']],
    mode='markers',
    name='Casing 13 3/8"',
    marker=dict(
        size=12,
        color='orange',      # Color naranja para este casing
        symbol='diamond',
        line=dict(color='black', width=2)  # Borde negro para que destaque
    ),
    hovertemplate='<b>Casing 13 3/8"</b><br>MD: ~2,500 pies<extra></extra>'
))

print("✅ Casing 13 3/8\" marcado en los tres pozos")
print(f"   Alfa 1: MD real = {punto_alfa1_13['MD']:.0f} pies")
print(f"   Alfa 2: MD real = {punto_alfa2_13['MD']:.0f} pies")
print(f"   Alfa 3: MD real = {punto_alfa3_13['MD']:.0f} pies")

## Celda 11: Agregar casing de 9 5/8" en los tres pozos

Ahora repetimos el mismo proceso para el casing intermedio de 9 5/8", que está a aproximadamente 8,500 pies de profundidad.

In [ ]:
# POZO ALFA 1 - Casing 9 5/8"
diferencias_alfa1_9 = abs(alfa1['MD'] - casing_9_58)
indice_alfa1_9 = diferencias_alfa1_9.idxmin()
punto_alfa1_9 = alfa1.loc[indice_alfa1_9]

# POZO ALFA 2 - Casing 9 5/8"
diferencias_alfa2_9 = abs(alfa2['MD'] - casing_9_58)
indice_alfa2_9 = diferencias_alfa2_9.idxmin()
punto_alfa2_9 = alfa2.loc[indice_alfa2_9]

# POZO ALFA 3 - Casing 9 5/8"
diferencias_alfa3_9 = abs(alfa3['MD'] - casing_9_58)
indice_alfa3_9 = diferencias_alfa3_9.idxmin()
punto_alfa3_9 = alfa3.loc[indice_alfa3_9]

# Agregar a la figura
fig.add_trace(go.Scatter3d(
    x=[punto_alfa1_9['NS'], punto_alfa2_9['NS'], punto_alfa3_9['NS']],
    y=[punto_alfa1_9['EW'], punto_alfa2_9['EW'], punto_alfa3_9['EW']],
    z=[-punto_alfa1_9['TVD'], -punto_alfa2_9['TVD'], -punto_alfa3_9['TVD']],
    mode='markers',
    name='Casing 9 5/8"',
    marker=dict(
        size=11,
        color='purple',      # Color morado para este casing
        symbol='diamond',
        line=dict(color='black', width=2)
    ),
    hovertemplate='<b>Casing 9 5/8"</b><br>MD: ~8,500 pies<extra></extra>'
))

print("✅ Casing 9 5/8\" marcado en los tres pozos")
print(f"   Alfa 1: MD real = {punto_alfa1_9['MD']:.0f} pies")
print(f"   Alfa 2: MD real = {punto_alfa2_9['MD']:.0f} pies")
print(f"   Alfa 3: MD real = {punto_alfa3_9['MD']:.0f} pies")

## Celda 12: Agregar casing de 7" en los tres pozos

Finalmente, agregamos el casing de producción de 7", que es el más profundo y llega casi hasta el fondo del pozo.

In [ ]:
# POZO ALFA 1 - Casing 7"
diferencias_alfa1_7 = abs(alfa1['MD'] - casing_7)
indice_alfa1_7 = diferencias_alfa1_7.idxmin()
punto_alfa1_7 = alfa1.loc[indice_alfa1_7]

# POZO ALFA 2 - Casing 7"
diferencias_alfa2_7 = abs(alfa2['MD'] - casing_7)
indice_alfa2_7 = diferencias_alfa2_7.idxmin()
punto_alfa2_7 = alfa2.loc[indice_alfa2_7]

# POZO ALFA 3 - Casing 7"
diferencias_alfa3_7 = abs(alfa3['MD'] - casing_7)
indice_alfa3_7 = diferencias_alfa3_7.idxmin()
punto_alfa3_7 = alfa3.loc[indice_alfa3_7]

# Agregar a la figura
fig.add_trace(go.Scatter3d(
    x=[punto_alfa1_7['NS'], punto_alfa2_7['NS'], punto_alfa3_7['NS']],
    y=[punto_alfa1_7['EW'], punto_alfa2_7['EW'], punto_alfa3_7['EW']],
    z=[-punto_alfa1_7['TVD'], -punto_alfa2_7['TVD'], -punto_alfa3_7['TVD']],
    mode='markers',
    name='Casing 7"',
    marker=dict(
        size=11,
        color='brown',       # Color café para este casing
        symbol='diamond',
        line=dict(color='black', width=2)
    ),
    hovertemplate='<b>Casing 7"</b><br>MD: ~10,300 pies<extra></extra>'
))

print("✅ Casing 7\" marcado en los tres pozos")
print(f"   Alfa 1: MD real = {punto_alfa1_7['MD']:.0f} pies")
print(f"   Alfa 2: MD real = {punto_alfa2_7['MD']:.0f} pies")
print(f"   Alfa 3: MD real = {punto_alfa3_7['MD']:.0f} pies")

## Celda 13: Marcar la Formación Hollín Superior

Ahora vamos a marcar las formaciones reservorio. Empezamos con el Hollín Superior, que es la parte superior de la zona productora de petróleo.

Usaremos círculos de color verde lima para distinguirlos de los diamantes de los casings.

In [ ]:
# POZO ALFA 1 - Hollín Superior
dif_alfa1_hsup = abs(alfa1['MD'] - hollin_superior)
idx_alfa1_hsup = dif_alfa1_hsup.idxmin()
punto_alfa1_hsup = alfa1.loc[idx_alfa1_hsup]

# POZO ALFA 2 - Hollín Superior
dif_alfa2_hsup = abs(alfa2['MD'] - hollin_superior)
idx_alfa2_hsup = dif_alfa2_hsup.idxmin()
punto_alfa2_hsup = alfa2.loc[idx_alfa2_hsup]

# POZO ALFA 3 - Hollín Superior
dif_alfa3_hsup = abs(alfa3['MD'] - hollin_superior)
idx_alfa3_hsup = dif_alfa3_hsup.idxmin()
punto_alfa3_hsup = alfa3.loc[idx_alfa3_hsup]

# Agregar a la figura
fig.add_trace(go.Scatter3d(
    x=[punto_alfa1_hsup['NS'], punto_alfa2_hsup['NS'], punto_alfa3_hsup['NS']],
    y=[punto_alfa1_hsup['EW'], punto_alfa2_hsup['EW'], punto_alfa3_hsup['EW']],
    z=[-punto_alfa1_hsup['TVD'], -punto_alfa2_hsup['TVD'], -punto_alfa3_hsup['TVD']],
    mode='markers',
    name='Hollín Superior',
    marker=dict(
        size=13,
        color='lime',        # Verde lima brillante
        symbol='circle',     # Círculo para distinguir de los casings
        line=dict(color='black', width=2)
    ),
    hovertemplate='<b>Hollín Superior</b><br>Reservorio<br>MD: ~10,167 pies<extra></extra>'
))

print("✅ Formación Hollín Superior marcada en los tres pozos")
print(f"   Alfa 1: MD real = {punto_alfa1_hsup['MD']:.0f} pies")
print(f"   Alfa 2: MD real = {punto_alfa2_hsup['MD']:.0f} pies")
print(f"   Alfa 3: MD real = {punto_alfa3_hsup['MD']:.0f} pies")

## Celda 14: Marcar la Formación Hollín Inferior

Finalmente, marcamos el Hollín Inferior, que típicamente es de mejor calidad que el Superior y es el objetivo final de estos pozos.

Usaremos un verde más oscuro para diferenciarlo del Hollín Superior.

In [ ]:
# POZO ALFA 1 - Hollín Inferior
dif_alfa1_hinf = abs(alfa1['MD'] - hollin_inferior)
idx_alfa1_hinf = dif_alfa1_hinf.idxmin()
punto_alfa1_hinf = alfa1.loc[idx_alfa1_hinf]

# POZO ALFA 2 - Hollín Inferior
dif_alfa2_hinf = abs(alfa2['MD'] - hollin_inferior)
idx_alfa2_hinf = dif_alfa2_hinf.idxmin()
punto_alfa2_hinf = alfa2.loc[idx_alfa2_hinf]

# POZO ALFA 3 - Hollín Inferior
dif_alfa3_hinf = abs(alfa3['MD'] - hollin_inferior)
idx_alfa3_hinf = dif_alfa3_hinf.idxmin()
punto_alfa3_hinf = alfa3.loc[idx_alfa3_hinf]

# Agregar a la figura
fig.add_trace(go.Scatter3d(
    x=[punto_alfa1_hinf['NS'], punto_alfa2_hinf['NS'], punto_alfa3_hinf['NS']],
    y=[punto_alfa1_hinf['EW'], punto_alfa2_hinf['EW'], punto_alfa3_hinf['EW']],
    z=[-punto_alfa1_hinf['TVD'], -punto_alfa2_hinf['TVD'], -punto_alfa3_hinf['TVD']],
    mode='markers',
    name='Hollín Inferior',
    marker=dict(
        size=13,
        color='darkgreen',   # Verde oscuro para distinguirlo del Superior
        symbol='circle',
        line=dict(color='black', width=2)
    ),
    hovertemplate='<b>Hollín Inferior</b><br>Reservorio<br>MD: ~10,212 pies<extra></extra>'
))

print("✅ Formación Hollín Inferior marcada en los tres pozos")
print(f"   Alfa 1: MD real = {punto_alfa1_hinf['MD']:.0f} pies")
print(f"   Alfa 2: MD real = {punto_alfa2_hinf['MD']:.0f} pies")
print(f"   Alfa 3: MD real = {punto_alfa3_hinf['MD']:.0f} pies")

## Celda 15: Configurar el aspecto de la figura

Ya tenemos todos los elementos en la figura, pero ahora necesitamos configurar cómo se ve: el título, los nombres de los ejes, el tamaño de la ventana, la leyenda, etc.

También configuramos la cámara inicial, que es el ángulo desde el cual vamos a ver la figura cuando se abra por primera vez.

In [ ]:
# Configuramos el layout (aspecto) de la figura
fig.update_layout(
    # Título principal de la figura
    title=dict(
        text='<b>TRES POZOS DEL CAMPO SACHA - ALFA 1, ALFA 2, ALFA 3</b>',
        x=0.5,               # Centrado horizontalmente
        xanchor='center',
        font=dict(size=20, color='darkblue')
    ),
    
    # Configuración de la escena 3D
    scene=dict(
        # Eje X (Norte-Sur)
        xaxis=dict(
            title='Desplazamiento Norte-Sur (pies)',
            title_font=dict(size=14),
            backgroundcolor='lightgray',
            gridcolor='white'
        ),
        # Eje Y (Este-Oeste)
        yaxis=dict(
            title='Desplazamiento Este-Oeste (pies)',
            title_font=dict(size=14),
            backgroundcolor='lightgray',
            gridcolor='white'
        ),
        # Eje Z (Profundidad)
        zaxis=dict(
            title='Profundidad TVD (pies)',
            title_font=dict(size=14),
            backgroundcolor='lightgray',
            gridcolor='white'
        ),
        # Aspecto de la figura (qué tan estirada se ve)
        aspectmode='manual',
        aspectratio=dict(
            x=1,     # Ancho horizontal NS
            y=1,     # Ancho horizontal EW
            z=1.5    # Altura (más alto para que se vea bien la profundidad)
        ),
        # Posición inicial de la cámara
        camera=dict(
            eye=dict(
                x=1.5,   # Posición X de la cámara
                y=1.5,   # Posición Y de la cámara
                z=1.2    # Posición Z de la cámara (altura)
            )
        )
    ),
    
    # Tamaño de la ventana de visualización
    height=900,    # Alto en píxeles
    width=1400,    # Ancho en píxeles
    
    # Configuración de la leyenda
    showlegend=True,
    legend=dict(
        x=1.02,    # Posición a la derecha de la figura
        y=0.98,    # Posición en la parte superior
        bgcolor='rgba(255,255,255,0.9)',  # Fondo blanco semi-transparente
        bordercolor='black',
        borderwidth=2,
        font=dict(size=12)
    ),
    
    # Fondo general
    paper_bgcolor='white'
)

print("✅ Layout configurado")
print("   La figura está lista para mostrarse")

## Celda 16: ¡Mostrar la visualización 3D!

Finalmente, mostramos la figura completa con todos sus elementos.

**Cómo interactuar con la visualización:**

- **Rotar:** Haz clic izquierdo y arrastra el mouse para girar la vista en cualquier dirección
- **Zoom:** Usa la rueda del mouse para acercar o alejar la vista
- **Pan (mover):** Haz clic derecho y arrastra para mover la vista sin rotarla
- **Hover:** Pasa el mouse sobre cualquier línea o punto para ver información detallada
- **Leyenda:** Haz clic en cualquier elemento de la leyenda para ocultarlo o mostrarlo
- **Reset:** Si te pierdes, usa el botón "Reset axes" que aparece cuando pasas el mouse sobre la figura

La visualización es completamente interactiva, así que explora libremente para entender mejor la geometría de los tres pozos.

In [ ]:
# Mostramos la figura
fig.show()

# Imprimimos un resumen de lo que se muestra
print("\n" + "="*70)
print("✅ VISUALIZACIÓN 3D COMPLETADA")
print("="*70)

print("\n📊 ELEMENTOS EN LA FIGURA:")
print("\n  TRAYECTORIAS:")
print("    • Pozo Alfa 1 (Rojo)")
print("    • Pozo Alfa 2 (Azul)")
print("    • Pozo Alfa 3 (Verde)")

print("\n  CASING POINTS:")
print("    • 13 3/8\" Surface (Naranja)")
print("    • 9 5/8\" Intermediate (Morado)")
print("    • 7\" Production (Café)")

print("\n  FORMACIONES:")
print("    • Hollín Superior (Verde Lima)")
print("    • Hollín Inferior (Verde Oscuro)")

print("\n💡 TIPS:")
print("   • Arrastra con el mouse para rotar la vista")
print("   • Usa la rueda para hacer zoom")
print("   • Pasa el mouse sobre los elementos para ver detalles")
print("   • Haz clic en la leyenda para ocultar/mostrar elementos")

print("\n¡Explora la visualización libremente!")

## Celda 17 (Opcional): Guardar la figura como archivo HTML

Si quieres guardar esta visualización para abrirla más tarde sin tener que ejecutar todo el notebook de nuevo, puedes guardarla como un archivo HTML.

El archivo HTML contendrá toda la visualización interactiva y podrás abrirlo en cualquier navegador web, compartirlo con colegas, o incluirlo en presentaciones.

In [ ]:
# Guardamos la figura como archivo HTML
nombre_archivo = 'tres_pozos_campo_sacha_3d.html'
fig.write_html(nombre_archivo)

print(f"✅ Figura guardada como '{nombre_archivo}'")
print("\n   Puedes abrir este archivo en cualquier navegador web")
print("   La visualización será completamente interactiva")
print("   ¡No necesitas Python para verla!")

---

## 🎉 ¡Felicidades!

Has completado exitosamente la visualización 3D de los tres pozos del Campo Sacha.

### ¿Qué aprendimos?

1. **Cómo cargar y procesar datos de surveys** de pozos petroleros desde archivos CSV
2. **Cómo crear visualizaciones 3D interactivas** usando Plotly
3. **Cómo encontrar puntos específicos** en un survey (casing points, formaciones)
4. **Cómo trabajar con sistemas de coordenadas 3D** para pozos direccionales
5. **Cómo personalizar y configurar figuras** para hacerlas profesionales y fáciles de entender

### Próximos pasos

Ahora que tienes esta base, puedes:
- Agregar más pozos a la visualización
- Incluir otras formaciones geológicas
- Cambiar los colores y estilos según tus preferencias
- Crear análisis comparativos entre los pozos
- Calcular distancias y desviaciones entre pozos

**¡El código es tuyo para modificar y experimentar!**